### ABSA via gpt-3.5-turbo

uses the cheaper model, but this is zero-shot and hence less consistent in formatting the output as JSON

this file is self-contained and does not require any external files (other than your csv)

original inspiration from Twitter, expanded with lots of suggestions by ChatGPT

In [ ]:
import os
import openai
import subprocess
import requests
from typing import List, Optional
from pprint import pprint
os.environ["OPENAI_API_KEY"] = "" 

API_KEY = os.getenv("OPENAI_API_KEY")
#GPT_MODEL = ["text-davinci-003"]

def print_response(response):
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        error = response.json()
        print(f"Error: {error['error']}")
        return error


In [ ]:
import pandas

In [ ]:
import os 
import logging
import sys

# Setup OPENAI_API_KEY

# Setup logging

log = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)s | %(levelname)s | %(message)s", level=logging.INFO)

# Update sys.path (or use PYTHONPATH)

#sys.path.insert(0, '..')

In [ ]:
from random import choice
from tqdm.notebook import tqdm
#from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent

analysis_results = []
extra_prompts = []

logging.getLogger("openai").setLevel(logging.INFO)

## Create completion
- DOC url: https://platform.openai.com/docs/api-reference/completions/create

# Making requests

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

response = requests.post("https://api.openai.com/v1/completions", \
    headers=headers, json={
        "model": "gpt-3.5-turbo",
        "prompt": "Say this is a test",
        "temperature": 0.2,
        "max_tokens": 1000
    }
)

def create_completion(messages:str, suffix:str = None, model: str = "gpt-3.5-turbo", max_tokens: Optional[int] = 500, temperature: float = 1, top_p: Optional[int] = 1, n: Optional[int] = 1,  stream: Optional[bool] = False, echo: Optional[bool] = False):
    """
    https://platform.openai.com/docs/api-reference/completions/create
    ##### Given a prompt, the model will return one or more predicted completions, and can also return the probabilities of alternative tokens at each position.

    Args:
        - prompt (str): 
            - The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays. 
            Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document. 
        
        - suffix (str): 
            - The suffix that comes after a completion of inserted text.
        
        - model (str, required): 
            - ID of the model to use. You can use the **List models** API to see all of your available models, or see our **Model overview** for descriptions of them. Defaults to "text-davinci-003".
        
        - max_tokens (int, optional, Default: 500): 
            - The maximum number of tokens to generate in the completion.
                The token count of your prompt plus `max_tokens` cannot exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Defaults to 500.
        
        - temperature (float, optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            We generally recommend altering this or `top_p` but not both. Defaults to 0.
            
        - top_p (float, optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.
            
        - n (int, optional, Default: 1):
            - The number of completions to generate. Defaults to 1. 
        
        - stream (bool, optional): 
            - Whether to stream back partial progress. If set, tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a `data: [DONE]` message. Defaults to False.
        
        - echo (bool, optional): _description_. Defaults to False.
        

    Returns:
        _type_: _description_
    """    
    response = requests.post("https://api.openai.com/v1/completions", \
        headers=headers, json={
            "model": model,
            "messages": messages,
            "suffix": suffix,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": stream,
            "echo": echo,
            "stop": suffix,
            "n": n,
            "top_p": top_p
        }
    )
    return response
    
def edit_prompt(instruction: str, input: Optional[str] = None, model: str = "text-davinci-edit-001", num: Optional[int] = 1, temperature: Optional[float] = 1, top_p: Optional[float] = 1):
    """#### Given a prompt and an instruction, the model will return an edited version of the prompt.
    https://platform.openai.com/docs/api-reference/edits
    Args:
        - instruction (str, required): 
            - The instruction that tells the model how to edit the prompt.
            
        - input (Optional[str], optional): 
            - The input text to use as a starting point for the edit. Defaults to None.
        
        - model (str, required): 
            - ID of the model to use. You can use the `text-davinci-edit-001` or `code-davinci-edit-001` model with this endpoint. Defaults to "text-davinci-edit-001".

        - n (Optional[int], optional): 
            - How many edits to generate for the input and instruction. Defaults to 1.

        - temperature (Optional[float], optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            - We generally recommend altering this or `top_p` but not both. Defaults to 1.
        
        - top_p (Optional[float], optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.

    Returns:
        - response
    """    
    response = requests.post("https://api.openai.com/v1/edits", \
        headers=headers, json={
            "model": model,
            "input": input,
            "instruction": instruction,
            "n": num,
            "temperature": temperature,
            "top_p": top_p
        }
    )
    return response

## Testing

In [ ]:
import csv
import pandas as pd
labels=pd.read_csv("./Ingeborg-Bachmann-Preis_Twitter_2014.csv",quotechar='"', escapechar='\\', error_bad_lines=False, doublequote=True, sep=";", encoding="utf-8")
df=labels[~labels.duplicated(subset="id",keep="first")]

In [ ]:
df.head()

In [ ]:
len(df.index)

In [ ]:
df['title'] = df['title'].astype(str)
df_duplicates = df[df.duplicated(subset='title', keep=False)]

In [ ]:
print(df_duplicates)

In [ ]:
print(labels.columns)

In [ ]:
#adjusting for ids that weren't unique
import csv
import pandas as pd
labels=pd.read_csv("./2011-2014/jury_output2015def_transformed.csv",quotechar='"', doublequote=True, sep="|", encoding="utf-8")
#labels['title'] = labels['permalink'].str.split('/').str[-1] #dit was nodig omdat de ids niet uniek waren in scientific notation
len(labels)
print(labels.head())
df=labels
#df=labels[~labels.duplicated(subset="id",keep="first")]

In [ ]:
#df.rename(columns = {'Document':'title', 'Title':'text'}, inplace = True)
df=labels
df.rename(columns = {'result_index':'title'}, inplace = True)

In [ ]:
#this one works, but forgot what is specific about this attempt
import time 
import traceback
from textwrap import dedent
from random import choice
from tqdm.notebook import tqdm
#from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent
import time
from json import JSONDecodeError, loads
from tqdm import tqdm

retry_attempts = 3
retry_delay = 2
analysis_results = []
extra_prompts = []

completion_text_conversation = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Gmünder, Kegel, Kastberger, Keller, Feßmann, Steiner needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Dinić, Dorian, Gardi, Lehn, Macht, Özdogan, Otoo, Sargnagel, Schenk, Schneider, Snela, Sozio, Wolf, Zwicky. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2007 = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Sulzer, Feßmann, Spinnen, Jandl, Fleischanderl, Winkels, Keller, Mangold, Strigl, Heiz, März, Nüchtern, Ebel, Rakusa, Corino, Radisch needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Janesch, Altwasser, Kloeble, Mezger, Elmiger, Ballhausen, Scharnigg, Scholz, Zander, Kleindienst, Wawerzinek, Schmidt, Fries, Rossbacher, Langenegger, Weiss, Krampitz, Preisendörfer, Neudecker, Stift, Bönt, Ruch, Petersen, Schäfer, Sander, Winkler, Born, Satanik, Palzhoff, Bronsky, Setz, Reitzer, Arndt, Findeis, Orths, Geißler, Mohafez, Hintze, Lenz, Ziegler, Rammstedt, Selg, Marinič, Bernhardt, Schmidt, Grill, Albrecht, Schley, Seiler, Scheuermann, Reng, Zwicky, Stavarič, Oda, Oesterle, PeterLicht, Böttcher, Kern, Stangl, Becker. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target in each sentence separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2014 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Dusini, Feßmann, Steiner, Winkels, Strigl, Spinnen, Sulzer, Jandl, Caduff, Keller needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn, Moster, Ramnek, Richner, Stichmann, Hassinger, Mahlke, Travnicek, Martynova, Kränzler, Froehling, Nawrat, Senkel, Federmair, Feimer, Geltinger, Steinbeis, Wisser, Praßler, Baum, Reichlin, Haderlap, Rabinowich, Bußmann, Popp, Randt, Richter, Božiković, Klupp. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" and their values as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2015 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Strigl, Spinnen, Dusini, Feßmann, Steiner, Keller, Jandl needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising as key-value pairs \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. If a single entity mentioned in the text seems to belong to multiple aspect categories and terms, select the most likely aspect category and aspect term out of the list.\n\n
"""
logging.getLogger("openai").setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)
for i, row in tqdm(df.iterrows(), total=len(df), desc="Analyzing reviews"):
    try:
        if pd.notnull(row["title"]):
            title = row["title"]
            text = row["text"]
            log.info(f"Analyzing feedback - \nID: {row['title']}\nTitle: {title}\nText: {text}\n")

            res = create_completion(
                model="gpt-3.5-turbo",
                temperature="0.2",
                messages=[
                    {"role": "system",    "content": "You are a data analyst that provides multi-target quadruple sentiment extraction."},
                    {"role": "system",    "content": "You annotate multiple opinion targets per line if the text contains more than one target. You annotate every single target only once. Do not invent aspect categories or aspect terms yourself."},
                    {"role": "user",      "content": completion_text_conversation2015 + text + ":"},    
                    #{"role": "user",      "content": "What is the most likely "},
                    #{"role": "user",      "content": "Where are most of the students from?"},
                    #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
                    # add in as many questions as you want
                ],
            )

            try:
                raw_json = res.json()["choices"][0]["message"]["content"].strip()
                #print(raw_json)
                json_data = loads(raw_json)
                analysis_results.append(json_data)

                log.debug(f"JSON response: {pprint(json_data)}")

                extra_prompts.append(f"\n{text}\n{raw_json}")

            except JSONDecodeError as e:
                if "Unfortunately, it seems that there is no text provided in your request for me to perform sentiment extraction on." in str(e):
                    log.error("No text provided for sentiment analysis")
                    analysis_results.append("No text provided for sentiment analysis")
                elif "Expecting value" in str(e):
                    if "Unterminated string" in str(e):
                        log.warning("JSON string is unterminated")
                        analysis_results.append(raw_json)
                    elif "I'm sorry, but I need some text to perform the sentiment extraction" in str(e):
                        log.error("No text provided for sentiment analysis")
                        analysis_results.append("No text provided for sentiment analysis")
                    else:
                        log.error(f"Failed to parse '{raw_json}' -> {e}")
                        analysis_results.append([])
                else:
                    log.error(f"Failed to parse '{raw_json}' -> {e}")
                    analysis_results.append([])
            except Exception as e:
                log.error(f"Error performing sentiment analysis on text: {text}")
                log.error(str(e))
                analysis_results.append([])
        else:
            log.warning(f"No id found in row {i}. Skipping analysis.")
            analysis_results.append([])
    except Exception as e:
        traceback.print_exc()
        log.warning(f"The error ecountered, {i}. Skipping analysis.")
        analysis_results.append([])
        
    #time.sleep(2) #added 2 seconds delay between requests

df["analysis"] = analysis_results
df.to_csv("./feedbacks_analysis_2015_jury_batch1.csv", index=False)

In [ ]:
#filehandling stuff exceptions (my Ids were not unique in scientific notation)
import csv
import pandas as pd
labels=pd.read_csv("./2011-2014/jury_output2015def_transformed.csv",quotechar='"', doublequote=True, sep="|", encoding="utf-8")
labels['title'] = labels['permalink'].str.split('/').str[-1] #dit was nodig omdat de ids niet uniek waren in scientific notation
len(labels)
print(labels.head())
df=labels
#df=labels[~labels.duplicated(subset="id",keep="first")]

In [ ]:
df=labels[~labels.duplicated(subset="title",keep="first")]

In [ ]:
#df.rename(columns = {'Document':'title', 'Title':'text'}, inplace = True)
df=labels
df.rename(columns = {'id':'title'}, inplace = True)

In [ ]:
len(df)

In [ ]:
ids = df["title"]
len(df[ids.isin(ids[ids.duplicated()])].sort_values("title"))

In [ ]:
df.iloc[1500]

### Putting everything together

In [ ]:
#putting everything together for testing
import time 
from textwrap import dedent
from random import choice
from tqdm.notebook import tqdm
from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent
import time
from json import JSONDecodeError, loads
from tqdm import tqdm

retry_attempts = 3
retry_delay = 2
analysis_results = []
extra_prompts = []
#
completion_text_conversation = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Gmünder, Kegel, Kastberger, Keller, Feßmann, Steiner needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Dinić, Dorian, Gardi, Lehn, Macht, Özdogan, Otoo, Sargnagel, Schenk, Schneider, Snela, Sozio, Wolf, Zwicky. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2007 = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Sulzer, Feßmann, Spinnen, Jandl, Fleischanderl, Winkels, Keller, Mangold, Strigl, Heiz, März, Nüchtern, Ebel, Rakusa, Corino, Radisch needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Janesch, Altwasser, Kloeble, Mezger, Elmiger, Ballhausen, Scharnigg, Scholz, Zander, Kleindienst, Wawerzinek, Schmidt, Fries, Rossbacher, Langenegger, Weiss, Krampitz, Preisendörfer, Neudecker, Stift, Bönt, Ruch, Petersen, Schäfer, Sander, Winkler, Born, Satanik, Palzhoff, Bronsky, Setz, Reitzer, Arndt, Findeis, Orths, Geißler, Mohafez, Hintze, Lenz, Ziegler, Rammstedt, Selg, Marinič, Bernhardt, Schmidt, Grill, Albrecht, Schley, Seiler, Scheuermann, Reng, Zwicky, Stavarič, Oda, Oesterle, PeterLicht, Böttcher, Kern, Stangl, Becker. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target in each sentence separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2014 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Dusini, Feßmann, Steiner, Winkels, Strigl, Spinnen, Sulzer, Jandl, Caduff, Keller needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn, Moster, Ramnek, Richner, Stichmann, Hassinger, Mahlke, Travnicek, Martynova, Kränzler, Froehling, Nawrat, Senkel, Federmair, Feimer, Geltinger, Steinbeis, Wisser, Praßler, Baum, Reichlin, Haderlap, Rabinowich, Bußmann, Popp, Randt, Richter, Božiković, Klupp. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising as key-value pairs \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2015 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Strigl, Spinnen, Dusini, Feßmann, Steiner, Keller, Jandl needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising as key-value pairs \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
logging.getLogger("openai").setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)


for i in tqdm(range(len(df)), desc="Analyzing reviews"):
    if pd.notnull(df.loc[i, "title"]):
        title = df.loc[i, "title"]
        text = df.loc[i, "text"]
        log.info(f"Analyzing feedback - \nID: {df.loc[i, 'title']}\nTitle: {title}\nText: {text}\n")

        res = create_completion(
            model="gpt-3.5-turbo",
            temperature="0.2",
            messages=[
                {"role": "system",    "content": "You are a data analyst that provides multi-target quadruple sentiment extraction."},
                {"role": "system",    "content": "You annotate multiple opinion targets per line if the text contains more than one target. You annotate every single target only once. Hashtags and weblinks are not targets. Do not invent aspect categories or aspect terms yourself."},
                {"role": "user",      "content": completion_text_conversation2015 + text + ":"},    
                #{"role": "user",      "content": "What is the most likely "},
                #{"role": "user",      "content": "Where are most of the students from?"},
                #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
                # add in as many questions as you want
            ],
        )

        try:
            raw_json = res.json()["choices"][0]["message"]["content"].strip()
            #print(raw_json)
            json_data = loads(raw_json)
            analysis_results.append(json_data)

            log.debug(f"JSON response: {pprint(json_data)}")

            extra_prompts.append(f"\n{text}\n{raw_json}")

        except JSONDecodeError as e:
            if "Unfortunately, it seems that there is no text provided in your request for me to perform sentiment extraction on." in str(e):
                log.error("No text provided for sentiment analysis")
                analysis_results.append("No text provided for sentiment analysis")
            elif "Expecting value" in str(e):
                if "Unterminated string" in str(e):
                    log.warning("JSON string is unterminated")
                    analysis_results.append(raw_json)
                elif "I'm sorry, but I need some text to perform the sentiment extraction" in str(e):
                    log.error("No text provided for sentiment analysis")
                    analysis_results.append("No text provided for sentiment analysis")
                else:
                    log.error(f"Failed to parse '{raw_json}' -> {e}")
                    analysis_results.append([])
            else:
                log.error(f"Failed to parse '{raw_json}' -> {e}")
                analysis_results.append([])
        except Exception as e:
            log.error(f"Error performing sentiment analysis on text: {text}")
            log.error(str(e))
            analysis_results.append([])
    else: #hier stond else
        traceback.print_exc()
        log.warning(f"No id found in row {i}. Skipping analysis.")
        analysis_results.append([])
        
    #time.sleep(2) #added 2 seconds delay between requests

df["analysis"] = analysis_results
df.to_csv("./feedbacks_analysis_2011-2014.csv", index=False)

In [ ]:
#FINAL VERSION
import time 
import traceback
from textwrap import dedent
from random import choice
from tqdm.notebook import tqdm
#from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent
import time
from json import JSONDecodeError, loads
from tqdm import tqdm

retry_attempts = 3
retry_delay = 2
analysis_results = []
extra_prompts = []

completion_text_conversation = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Gmünder, Kegel, Kastberger, Keller, Feßmann, Steiner needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Dinić, Dorian, Gardi, Lehn, Macht, Özdogan, Otoo, Sargnagel, Schenk, Schneider, Snela, Sozio, Wolf, Zwicky. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2007 = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Sulzer, Feßmann, Spinnen, Jandl, Fleischanderl, Winkels, Keller, Mangold, Strigl, Heiz, März, Nüchtern, Ebel, Rakusa, Corino, Radisch needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Janesch, Altwasser, Kloeble, Mezger, Elmiger, Ballhausen, Scharnigg, Scholz, Zander, Kleindienst, Wawerzinek, Schmidt, Fries, Rossbacher, Langenegger, Weiss, Krampitz, Preisendörfer, Neudecker, Stift, Bönt, Ruch, Petersen, Schäfer, Sander, Winkler, Born, Satanik, Palzhoff, Bronsky, Setz, Reitzer, Arndt, Findeis, Orths, Geißler, Mohafez, Hintze, Lenz, Ziegler, Rammstedt, Selg, Marinič, Bernhardt, Schmidt, Grill, Albrecht, Schley, Seiler, Scheuermann, Reng, Zwicky, Stavarič, Oda, Oesterle, PeterLicht, Böttcher, Kern, Stangl, Becker. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target in each sentence separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2014 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Dusini, Feßmann, Steiner, Winkels, Strigl, Spinnen, Sulzer, Jandl, Caduff, Keller needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn, Moster, Ramnek, Richner, Stichmann, Hassinger, Mahlke, Travnicek, Martynova, Kränzler, Froehling, Nawrat, Senkel, Federmair, Feimer, Geltinger, Steinbeis, Wisser, Praßler, Baum, Reichlin, Haderlap, Rabinowich, Bußmann, Popp, Randt, Richter, Božiković, Klupp. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" and their values as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
completion_text_conversation2015 = """
Please perform quadruple sentiment extraction on this utterance. The aspect categories and their aspect terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Strigl, Spinnen, Dusini, Feßmann, Steiner, Keller, Jandl needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Marchel, Preiwuß, Sommer, Klemm, Flor, Heier, Pölzl, Varatharajah, Fehr, Ganzoni, Gericke, Rubinowitz, Petz, Köhler, Boehning, Meyerhoff, Kegele, Güntner, Mueller, Rock, Simon, Helle, Schönthaler, Petrowskaja, Dübgen, Ehrlich, Maack, Mehlhorn. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a JSON object or a list of JSON objects comprising as key-value pairs \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" as list of dictionaries as the top-level objects. Do not add \"results\", \"aspect_extraction\" or other descriptive terms as top-level JSON object. The confidence score is represented as a string in the output. The order is the same as the order expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""
logging.getLogger("openai").setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)
for i, row in tqdm(df.iterrows(), total=len(df), desc="Analyzing reviews"):
    try:
        if pd.notnull(row["title"]):
            title = row["title"]
            text = row["text"]
            log.info(f"Analyzing feedback - \nID: {row['title']}\nTitle: {title}\nText: {text}\n")

            res = create_completion(
                model="gpt-3.5-turbo",
                temperature="0.2",
                messages=[
                    {"role": "system",    "content": "You are a data analyst that provides multi-target quadruple sentiment extraction."},
                    {"role": "system",    "content": "You annotate multiple opinion targets per line if the text contains more than one target. You annotate every single target only once. If the input text does not seem to contain any opinion targets or expressions, assume as opinion target the first noun and as expression the rest of the utterance. Hashtags and weblinks are not targets. Do not invent aspect categories or aspect terms yourself."},
                    {"role": "user",      "content": completion_text_conversation2015 + text + ":"},    
                    #{"role": "user",      "content": "What is the most likely "},
                    #{"role": "user",      "content": "Where are most of the students from?"},
                    #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
                    # add in as many questions as you want
                ],
            )

            try:
                raw_json = res.json()["choices"][0]["message"]["content"].strip()
                #print(raw_json)
                json_data = loads(raw_json)
                analysis_results.append(json_data)

                log.debug(f"JSON response: {pprint(json_data)}")

                extra_prompts.append(f"\n{text}\n{raw_json}")

            except JSONDecodeError as e:
                if "Unfortunately, it seems that there is no text provided in your request for me to perform sentiment extraction on." in str(e):
                    log.error("No text provided for sentiment analysis")
                    analysis_results.append("No text provided for sentiment analysis")
                elif "Expecting value" in str(e):
                    if "Unterminated string" in str(e):
                        log.warning("JSON string is unterminated")
                        analysis_results.append(raw_json)
                    elif "I'm sorry, but I need some text to perform the sentiment extraction" in str(e):
                        log.error("No text provided for sentiment analysis")
                        analysis_results.append("No text provided for sentiment analysis")
                    else:
                        log.error(f"Failed to parse '{raw_json}' -> {e}")
                        analysis_results.append([])
                else:
                    log.error(f"Failed to parse '{raw_json}' -> {e}")
                    analysis_results.append([])
            except Exception as e:
                log.error(f"Error performing sentiment analysis on text: {text}")
                log.error(str(e))
                analysis_results.append([])
        else:
            log.warning(f"No id found in row {i}. Skipping analysis.")
            analysis_results.append([])
    except Exception as e:
        traceback.print_exc()
        log.warning(f"The error ecountered, {i}. Skipping analysis.")
        analysis_results.append([])
        
    #time.sleep(2) #added 2 seconds delay between requests

df["analysis"] = analysis_results
df.to_csv("./feedbacks_analysis_2015_jury_batch1.csv", index=False)




### Emergency procedure 

In [ ]:
#disconnect emergency procedure
#print(analysis_results)
len(analysis)

In [ ]:
len(analysis_results) #shows the number of rows in the dataframe, insert this in the cell mentioning df.loc

In [ ]:
df.head(80)["analysis"] = analysis_results
df.to_csv("./feedbacks_analysis_2011-2014_0504_eerste80disconnect.csv", index=False)

In [ ]:
df.loc[df.head(80).index, "analysis"] = analysis_results #moans about deprecation, but the results are written
df.to_csv("./feedbacks_analysis_2011-2014_0504_eerste80disconnect_tweedemanier.csv", index=False) 

### Experiments to understand what the prompts do

In [ ]:
pprint(res_comp.json(), indent=2)

In [ ]:
import openai
#openai.api_key = "キー"
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    message=[        {            "role": "user",            "content": "質問"        }    ]
)
print(res["choices"][0]["text"].strip())


In [ ]:
import os
import openai
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  messages = [{"role": "system", "content" : "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-02"},
{"role": "user", "content" : "How are you?"},
{"role": "assistant", "content" : "I am doing well"},
{"role": "user", "content" : "What is the mission of the company OpenAI?"}]
)
#print(completion)
print(completion)

In [ ]:
completion_text = """
The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.

Human: Hello, who are you?
AI:
"""

res_comp = create_completion(completion_text, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)

In [ ]:
print(res_comp)

In [ ]:
completion_text_conversation = """
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""
res = create_completion(
    model="gpt-3.5-turbo",
    #message=[        {            "role": "user",            "content": "質問"        }    ]
#)
    #print(res["choices"][0]["text"].strip())
    messages=[
            {"role": "system",    "content": "You are a helpful bot"},
            #{"role": "system",    "content": prompt2},
            {"role": "user",      "content": completion_text_conversation},
            #{"role": "user",      "content": "What is most popular major or group of majors?"},
            #{"role": "user",      "content": "Where are most of the students from?"},
            #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
            # add in as many questions as you want
        ],
)
print(res.json()["choices"][0]["message"]["content"].strip())




In [ ]:
#for testing with individual examples without csv file
completion_text_conversation = """
Please classify the German text in terms of \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. Do not invent \"Aspect Category\" or \"Aspect Term\" yourself. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT:  Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: General\". Any reference to one of the names Winkels, Gmünder, Kegel, Kastberger, Keller, Feßmann, Steiner needs to be classified as \"JURY: Name\". Any reference to either one of the following names needs to be classified as \"CONTENDER: Name\": Dinić, Dorian, Gardi, Lehn, Macht, Özdogan, Otoo, Sargnagel, Schenk, Schneider, Snela, Sozio, Wolf, Zwicky. Any reference to any other named entity persons is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. Format output as JSON. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain.\n\n Lohnarbeit ist Demütigung - immer und überall. Das Buch ist wirklich scheiße. #tddl16 #sargnagel:
"""
#der Akzent des Autors ist so sympathisch. Aber dieser Juror, der da redet. Dieser selbstgefällige kleine Meckerer. Der fängt an, zu nerven. Wray performt zu viel mit den Händen. Das lenkt zu sehr vom Text ab #tddl Sargnagel hat eine technisch, intellektuell und literarisch hervorragende Geschichte abgeliefert. Statt Thomas Bernhard werden Parallelen zu Wolf Haas gezogen. Auch das ist wohltuend an der Diskussion:\n\n    [\n      {{ \"aspect category\": \"CONTENDER\", \"aspect term\": \"Appearance_Clothing\", \"opinion target\": \"Akzent des Autors\", \"opinion expression\": \"ist so sympathisch\", \"sentiment\": \"positive\", \"confidence score\": \"1\" }},\n      {{ \"aspect category\": \"JURY\", \"aspect term\": \"Behaviour\", \"opinion target\": \"dieser Juror, der da redet\", \"opinion expression\": \"Dieser selbstgefällige kleine Meckerer\", \"sentiment\": \"negative\", \"confidence score\": \"-1\" }},\n      {{ \"aspect category\": \"JURY\", \"aspect term\": \"Behaviour\", \"opinion target\": \"dieser Juror, der da redet\", \"opinion expression\": \"fängt an, zu nerven\", \"sentiment\": \"negative\", \"confidence score\": \"-1\" }},\n      {{ \"aspect category\": \"CONTENDER\", \"aspect term\": \"Appearance_Clothing\", \"opinion target\": \"Sargnagel\", \"opinion expression\": \"performt zu viel mit den Händen\", \"sentiment\": \"negative\", \"confidence score\": \"-0.7\" }},\n      {{ \"aspect category\": \"TEXT\", \"aspect term\": \"Language_Style\", \"opinion target\": \"Geschichte\", \"opinion expression\": \"technisch, intellektuell und literarisch hervorragende\", \"sentiment\": \"positive\", \"confidence score\": \"0.7\" }},\n      {{ \"aspect category\": \"ALLO-REFERENCE\", \"aspect term\": \"Other-Author\", \"opinion target\": \"werden Parallelen zu Wolf Haas gezogen\", \"opinion expression\": \"das ist wohltuend an der Diskussion\", \"sentiment\": \"positive\", \"confidence score\": \"0.5\" }}\n       {{ \"aspect category\": \"ALLO-REFERENCE\", \"aspect term\": \"Other-Author\", \"opinion target\": \"Thomas Bernhard \", \"opinion expression\": \"statt\", \"sentiment\": \"negative\", \"confidence score\": \"-0.1\" }}\n    ]\n\n    Der Text handelt vom Überleben in den Konzentrationslagern und wird aus der Sicht eines unzuverlässigen Erzählers erzählt. Der Text von Peschka hingegen: Beklemmend. Dumpf. Spröde. Karg. Verstörend. Irritierend. - Ein Endspiel Eingeschlossener mit Mäusen in ihren Bunkern. - Es fröstelt mich, trotz der Hitze im Text. Eindringlich. Geduldiges Warten im Saal. Ich finde die Jury heuer so zahm und alles irgendwie gutfindend. Einige Besucher nutzen den Text als Fächer. Wahrscheinlich das Beste, was man damit machen kann.:\n\n    [\n      {{ \"aspect category\": \"TEXT\", \"aspect term\": \"Motifs_Themes\", \"opinion target\": \"der Text\", \"opinion expression\": \"handelt vom Überleben in den Konzentrationslagern\", \"sentiment\": \"neutral\", \"confidence score\": \"1\" }},\n      {{ \"aspect category\": \"TEXT\", \"aspect term\": \"PoV_Narration\", \"opinion target\": \"der Text\", \"opinion expression\": \"wird aus der Sicht eines unzuverlässigen Erzählers erzählt\", \"sentiment\": \"neutral\", \"confidence score\": \"0\" }},\n      {{ \"aspect category\": \"TEXT\", \"aspect term\": \"Language_Style\", \"opinion target\": \"Der Text von Peschka\", \"opinion expression\": \"Beklemmend. Dumpf. Spröde. Karg. Verstörend. Irritierend. \", \"sentiment\": \"positiv\", \"confidence score\": \"0,3\" }},\n      {{ \"aspect category\": \"TEXT\", \"aspect term\": \"Motifs_Themes\", \"opinion target\": \"Eingeschlossener mit Mäusen in ihren Bunkern\", \"opinion expression\": \"ein Endspiel\", \"sentiment\": \"neutral\", \"confidence score\": \"0\" }},\n      {{ \"aspect category\": \"ONSITE-AUDIENCE\", \"aspect term\": \"Behaviour\", \"opinion target\": \"im Saal\", \"opinion expression\": \"Geduldiges Warten\", \"sentiment\": \"neutral\", \"confidence score\": \"0\" }}\n       {{ \"aspect category\": \"JURY\", \"aspect term\": \"valuation\", \"opinion target\": \"die Jury\", \"opinion expression\": \"heuer so zahm und alles irgendwie gutfindend\", \"sentiment\": \"negative\", \"confidence score\": \"-0,3\" }}\n       {{ \"aspect category\": \"ONSITE-AUDIENCE\", \"aspect term\": \"Behaviour\", \"opinion target\": \" Einige Besucher \", \"opinion expression\": \"nutzen den Text als Fächer.\", \"sentiment\": \"neutral\", \"confidence score\": \"0\" }}\n       {{ \"aspect category\": \"ONSITE-AUDIENCE\", \"aspect term\": \"Behaviour\", \"opinion target\": \"nutzen den Text als Fächer\", \"opinion expression\": \"das Beste, was man damit machen kann\", \"sentiment\": \"negative\", \"confidence score\": \"-0,3\" }}\n    ]\n\n 
res = create_completion(
    model="gpt-3.5-turbo",
    #message=[        {            "role": "user",            "content": "質問"        }    ]
#)
    #print(res["choices"][0]["text"].strip())
    messages=[
            {"role": "system",    "content": "You are a data analyst that analyzes complex datasets and provides multi-target quadruple sentiment extraction."},
            {"role": "system",    "content": "You annotate multiple opinion targets per line. Hashtags and weblinks are not targets."},
            {"role": "user",      "content": completion_text_conversation},
            #{"role": "user",      "content": "What is most popular major or group of majors?"},
            #{"role": "user",      "content": "Where are most of the students from?"},
            #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
            # add in as many questions as you want
        ],
)
#print(res)
print(res.json()["choices"][0]["message"]["content"].strip())

In [ ]:
#another test
completion_text_conversation = """
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""
res = create_completion(
    model="gpt-3.5-turbo",
    #message=[        {            "role": "user",            "content": "質問"        }    ]
#)
    #print(res["choices"][0]["text"].strip())
    messages=[
            {"role": "system",    "content": "You are a helpful bot"},
            #{"role": "system",    "content": prompt2},
            {"role": "user",      "content": completion_text_conversation},
            #{"role": "user",      "content": "What is most popular major or group of majors?"},
            #{"role": "user",      "content": "Where are most of the students from?"},
            #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
            # add in as many questions as you want
        ],
)
print(res.json()["choices"][0]["message"]["content"].strip())

In [ ]:
#pprint(res_comp)
    #raw_json = res_comp["choices"][0]["text"].strip() #res_comp ipv res
    pprint(res_comp.json()['choices'][0]['text'], indent=2)
    response_data = res_comp.json()
    print(response_data)
    raw_json = response_data["choices"][0]["text"].strip()

In [ ]:
pprint(res.json(), indent=2)

In [ ]:
import openai
import asyncio
from aiohttp import ClientSession

openai.api_key = "key_here"


openai.aiosession.set(ClientSession())

async def create_completion(prompt="How are you ?"):
    return await openai.ChatCompletion.acreate(messages=[{"role": "user", "content": prompt}], model="gpt-3.5-turbo")

async def main():

    return await asyncio.gather(*[create_completion() for _ in range(14)])

answers=await main()

await openai.aiosession.get().close()

answers

In [ ]:
completion_text_conversation = """
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""
res_comp = create_completion(completion_text_conversation, model="gpt-3.5-turbo", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json()['choices'][0]['text'], indent=2)

In [ ]:
completion_text_summarization = """
Summarize this for a second-grade student:

Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus.
"""
res_comp = create_completion(completion_text_summarization, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json()['choices'][0]['text'], indent=2)

In [ ]:
completion_text_episode = """
Write an episode for 2-minute speaking practice English under 100 words. This episode is for blog posts in markdown format. heading has 3 hashes. You are given main subject and subtopic as [main subject, subtopic]

[Amusement Park, Parade is more fun than rides]: 
"""
res_comp = create_completion(completion_text_episode, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
pprint(res_comp.json(), indent=2)

In [ ]:
completion_text = """
I just created a new Github repository. How can I connect this repository to my local git?

Description: 
"""
res_comp = create_completion(completion_text, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
print("".join(res_comp.json()['choices'][0]['text']))

In [ ]:
completion_text = """
I'd like to fix the following shell script. there are 5 zip files in the current directory. I want to unzip them into 5 different directories. The directory name should be added 7 to the original directory name. For example, '1월.zip' should be unzipped into '8월' directory.

#!/bin/bash

for i in $(seq -w 1 5); do
  unzip "$i.*.zip" -d "$i"
done


Description: 
"""
res_comp = create_completion(completion_text, model="text-davinci-003", max_tokens=500, temperature=0.2, top_p=1, n=1, stream=False, echo=False)
print("".join(res_comp.json()['choices'][0]['text']))

# Edit prompt

In [ ]:
example_code = """
How come I use my phone to check the weather? My teacher said that I should use.
"""

In [ ]:
edit_res = edit_prompt("Fix the sentence if the it looks awkward.", input=example_code, model="text-davinci-edit-001", num=1, temperature=0.2, top_p=1)
print(edit_res.json())
print(''.join(edit_res.json()['choices'][0]['text']))